# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
#HK1: Obtained all dependencies from second Assignment

import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import  TabularDatasetFactory

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.40.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#HK2: Get the workspace

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-final-project-hyperdrive'

experiment=Experiment(ws, experiment_name)

# start logging
run = experiment.start_logging


In [3]:
#HK3: Get the Cluster

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster

amlcompute_cluster_name = "assign3-cluster1"
amlcompute_cluster_name = "notebook195150"
# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

#compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)


Found existing cluster, use it.


In [17]:
#%%writefile conda_dependencies.yml

#dependencies:
#    scikit-learn
#    pip:
#        azureml-defaults

Overwriting conda_dependencies.yml


In [23]:
from azureml.core import Environment
sklearn_env = Environment.from_conda_specification(name='sklearn_env', file_path ='conda_dependencies.yml') 

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [24]:
from azureml.core import ScriptRunConfig
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice

#sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path ='conda_dependencies.yml') 

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    '--C' : choice(1,10),
    '--gamma': choice(0.2,2)
})

#TODO: Create your script config and hyperdrive config
script = ScriptRunConfig(
    source_directory = '.',
    script = 'train_heart.py',
    compute_target = compute_target,
    environment = sklearn_env,
    arguments = ['--C',1,'gamma',2]
)

hyperdrive_run_config = HyperDriveConfig(
    run_config = script,
    hyperparameter_sampling=param_sampling,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=21,
    max_concurrent_runs=4,
    policy=early_termination_policy)

In [25]:
#TODO: Submit your experiment
hyperdrive_submission = experiment.submit(hyperdrive_run_config)

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = <your policy here>

#TODO: Create the different params that you will be using during training
param_sampling = <your params here>

#TODO: Create your estimator and hyperdrive config
estimator = <your estimator here>

hyperdrive_run_config = <your config here>

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [27]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_submission).show()
hyperdrive_submission.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_fe55033f-cff4-4618-8fbb-3ad0c1f58c00
Web View: https://ml.azure.com/runs/HD_fe55033f-cff4-4618-8fbb-3ad0c1f58c00?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-195150/workspaces/quick-starts-ws-195150&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-05-10T08:03:33.704817][API][INFO]Experiment created<END>\n""<START>[2022-05-10T08:03:34.519270][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2022-05-10T08:03:35.4449378Z][SCHEDULER][INFO]Scheduling job, id='HD_fe55033f-cff4-4618-8fbb-3ad0c1f58c00_0'<END><START>[2022-05-10T08:03:35.5608108Z][SCHEDULER][INFO]Scheduling job, id='HD_fe55033f-cff4-4618-8fbb-3ad0c1f58c00_1'<END><START>[2022-05-10T08:03:35.6725649Z][SCHEDULER][INFO]Scheduling job, id='HD_fe55033f-cff4-4618-8fbb-3ad0c1f58c00_2'<END>"<START>[2022-05-10T08:03:35.709925][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to 

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User errors were found in at least one of the child runs.",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"User errors were found in at least one of the child runs.\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#ikzelf
# HK-Step 6: Retrieve and save your best automl model.

best_run_auto, best_fit_model = remote_run.get_output()

print(best_run_auto)


print("best-run details :", best_run_auto.get_details())
print('\n')
print("best-run id : ",best_run_auto.id)
print('\n')
print("best-run file names :", best_run_auto.get_file_names())
print('\n')
print("best-run metrics :", best_run_auto.get_metrics())



In [ ]:
best_run = hyperdrive_submission.get_best_run_by_primary_metric() #Best Run
best_run.get_details()
print('ID : ',best_run.id)
print('Metrics : ',best_run.get_metrics())

In [ ]:
#TODO: Save the best model

#HK7: Save the best model
import joblib

joblib.dump(best_fit_model, 'best_fit_model_hyperparameter.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

